In [61]:
import pandas as  pd
import numpy as np
from datetime import datetime 
from datetime import timedelta
import time
import ccxt
exchange = ccxt.ftx()

In [192]:
def get_position(symbols): 
    i= 0 
    while i <5 :
        try:
            res = exchange.fetchPositions()
            for sym in res:
                if sym['info']['future']==symbols:
                    f_pos = sym['info']
            return f_pos
        except Exception as e:
            if i >5:
                print(f'[{get_time()}]MAX RETRY {i} , {str(e)} Cannot GET {get_position.__name__}')
                return  None
            else:
                print(str(e) , get_position.__name__, f' RETRY {i}')
                time.sleep(5)
                i+=1


    

def get_ticker(symbol):
    i= 0
    while i< 5:
        try:
            res =exchange.fetch_ticker(symbol)
            return res
        except Exception as e:
            if i >5:
                print(f'[{get_time()}]MAX RETRY {i} , {str(e)} Cannot GET {get_ticker.__name__}')
                return  None
            else:
                print(str(e) , get_ticker.__name__, f' RETRY {i}')
                time.sleep(5)
                i+=1
    
    
def get_cash():
    try:
        wallet = get_wallet()
        for t in wallet:
            if t['coin'] in  ['USD']:


                cash = float(t['availableWithoutBorrow'])
        return cash
    except:
        return None
    

def get_unit( symbols):
    try:
        wallet = get_wallet()
        token_name = symbols.split('/')[0]
        unit = 0
        for token in wallet:
            if token['coin'] == token_name:
                valuex = float(token['availableWithoutBorrow'])
                unit += (valuex)

        return float(unit)
    except Exception as e:
        print(str(e))
        print('No Asset In Wallet')
        unit = 0
        return float(unit)

In [193]:
def _get_token_name( symbol):

    if len(symbol.split('/')) == 2:
        token_name = symbol.split('/')[0]
        token_namex = token_name.lower()

    if len(symbol.split('-')) == 2:
        token_name = symbol.split('/')[0]
        token_namex = token_name.lower()

    else :
        token_name = symbol
        token_namex = token_name.lower()
    return token_namex

def get_time():    
    now = datetime.now()
    formatted_date = now.strftime("%Y/%m/%d %H:%M:%S") 
    return formatted_date

def _hold_values( symbol):  ## [date,usd,unit,now_price]
    dt_now = get_time()
    asset_dict = {}
    asset_dicts = []
    sum_value = 0

    if symbol != 'all':
        # token_name = symbol.split('/')[0]
        unit_hold = get_unit(symbol)
        # last = get_price(symbol)
        last = get_ticker(symbol)['last']
        name = _get_token_name(symbol)
        usd_value = (unit_hold * last)
        #         usd_value = self.round_by_step(usd_value,0.01)
        asset_dict[symbol] = {'date': dt_now, f'unit': unit_hold,
                              f'usd_values': usd_value, 'symbol': symbol, 'name': name}
        cash = get_cash()
        asset_dict['USD'] = {'date': dt_now, f'unit': cash,
                             f'usd_values': cash, 'symbol': 'USD', 'name': 'usd'}
        sum_value += usd_value
        total_all_value = sum_value + cash
        msg = f'Check Value {name} + Cash {cash} = {total_all_value}'
        # log(msg)
        print(msg)

    elif symbol == 'all':
        res = exchange.privateGetWalletBalances()['result']
        for i in res:  ###
            # Check Values > 0 $
            valuex = float(i['usdValue'])
            if valuex > 0:
                # print(i)
                symbol  = i['coin'] + '/USD'
                name = _get_token_name(symbol)
                if i['coin'] != 'USD':
                    last = get_ticker(symbol)['last']
                    unit_hold = get_unit(symbol)

                    usd_value = (unit_hold * last)
                    asset_dict[symbol] = {'date': dt_now, 'unit': unit_hold,
                                     f'usd_values': usd_value, 'symbol': symbol, 'name': name}
                    sum_value += usd_value
                    # print(symbol,usd_value,last,unit_hold)

            cash = get_cash()
            asset_dict['USD'] = {'date': dt_now, f'unit': cash,
                                     f'usd_values': cash, 'symbol': 'USD', 'name': 'usd'}
        total_all_value = sum_value + cash


    return (asset_dict,total_all_value)



def equity_report():
    hold_all = _hold_values('all')
    port_dict = hold_all[0]
    total_equity = hold_all[1]
    equity = pd.DataFrame()
    
    equity['date'] = pd.Series(get_time())
    # token_name =[]

    for i in port_dict:
        if ('/USD' not in port_dict[i]['name']):
            name = port_dict[i]['name'].split('/')[0] ## Test and append
#             name = port_dict[i]['name']
            valuex = port_dict[i]['usd_values']
            valuex = float(valuex)
            if valuex >= 10:
                unitx = port_dict[i]['unit']
                exposure = valuex / total_equity
                equity[f'{name}_value'] = valuex
                equity[f'{name}_unit'] = unitx
                equity[f'{name}_exposure'] = valuex

    #         token_name.append(name)
    equity['usd'] = port_dict['USD']['usd_values']
    equity['equity'] = total_equity

    return equity


# Calculation Function

In [194]:
def Cal_SLdistance(df, vol_multiply = 0.2):
    vol = ta.STDDEV(df.close,30).ewm(alpha=0.96).mean()
    sl_distance = vol * vol_multiply
    return sl_distance.iloc[-1]

def Cal_TPdistance(df, vol_multiply = 1.2):
    vol = ta.STDDEV(df.close,30).ewm(alpha=0.96).mean()
    tp_distance = vol * vol_multiply
    return tp_distance.iloc[-1]

def Cal_Size(df, rpt :float):
    cash =get_cash()
    size= 0.0
    if cash != None:
        riskpertrade = rpt * cash
        size = (riskpertrade / Cal_SLdistance(df)) 
        if size > robot_position_size_limit:
            size = robot_position_size_limit
        elif size < robot_min_size:
            size = robot_min_size    
    return round(size, get_size_digit(robot_symbol))          

In [195]:
import os
import configparser
config = configparser.ConfigParser()
path_file = os.path.abspath('')
# print(path_file)
path_file = 'C:/Users/User\Desktop/sahing/crypto_anlysis'
config_file = os.path.join(path_file,'Future_Bot\\key_tao_main.ini')
config.read(config_file)

exchange = ccxt.ftx({'api_key':config['key']['apikey'],'secret':config['key']['secretkey'] ,'enableRateLimit': True,
                     'headers': {'FTX-SUBACCOUNT': 'TAO_MAIN',}})


In [230]:
        
def price_list(list_split,entry_price,mode='tp',side='buy'):
    sum_p = 0.0
    price_list = []
    if mode== 'tp':
        for i in list_split:
            sum_p +=i 
            if side== 'buy':
                tp_price =(entry_price * sum_p)+ entry_price
            elif side=='sell':
                tp_price = entry_price -(entry_price * sum_p)
            price_list.append(tp_price)
        # price_list = list(set(price_list))
        print('#'*100)
#     elif mode == 'sl':
    elif mode=='sl':
        for i in list_split:
            sum_p +=i 
            if side== 'buy':

                sl_price = entry_price - (entry_price * sum_p)
            elif side=='sell':
                sl_price = entry_price + (entry_price * sum_p)
               
            price_list.append(sl_price)
            # price_list = list(set(price_list))
        
    return price_list

        
tp_price_target = [0.05]
tp_price_list = price_list(tp_price_target,last_price,mode='tp')
tp_price_list

####################################################################################################


[0.19082700000000002]

In [231]:
sl_price_target = [0.10]
sl_price_list = price_list(sl_price_target,last_price,mode='sl',side='sell')
sl_price_list

[0.199914]

In [232]:
tp_size = []

In [233]:
# Test        
tp_split =tp_split_clean([0.05,0.02,0.03])
sl_split = sl_split_clean([0.02,0.03])



tp_price_list = price_list(tp_split,last_price,mode='tp')
sl_price_list = price_list(sl_split,last_price,mode='sl')

# print(tp_split,sl_split)
print(tp_price_list,sl_price_list)

####################################################################################################
[0.19082700000000002, 0.19446180000000002, 0.199914] [0.17810520000000002, 0.172653, 0.172653]


In [172]:
df1 =equity_report()
df1

,date,usd_value,usd_unit,usd_exposure,ftt_value,ftt_unit,ftt_exposure,eth_value,eth_unit,eth_exposure,usd,equity
0,2022/03/31 22:12:03,4994.827917,4994.827917,4994.827917,2474.0014,49.4,2474.0014,1425.3408,0.423,1425.3408,4994.827917,8894.170117


In [37]:
df1[df1.columns[df1.columns.str.endswith('_value')]]

,usd_value,ftt_value,eth_value
0,5163.729721,2462.096,1416.627


# Multi Symbol

In [276]:
symbol = 'ZIL-PERP'
all_position =[]

for resp in res:
    if float(resp['info']['size']) >0.0:
        all_position.append(resp['info'])

all_position

[{'future': 'FTT-PERP',
  'size': '61.8',
  'side': 'sell',
  'netSize': '-61.8',
  'longOrderSize': '0.0',
  'shortOrderSize': '0.0',
  'cost': '-3019.9188',
  'entryPrice': '48.866',
  'unrealizedPnl': '0.0',
  'realizedPnl': '-1.6686',
  'initialMarginRequirement': '0.33333333',
  'maintenanceMarginRequirement': '0.03',
  'openSize': '61.8',
  'collateralUsed': '1006.639589933604',
  'estimatedLiquidationPrice': '613.5693250439035',
  'recentAverageOpenPrice': '48.839',
  'recentPnl': '-1.6686',
  'recentBreakEvenPrice': '48.839',
  'cumulativeBuySize': '0.0',
  'cumulativeSellSize': '61.8'},
 {'future': 'ZIL-PERP',
  'size': '26620.0',
  'side': 'sell',
  'netSize': '-26620.0',
  'longOrderSize': '0.0',
  'shortOrderSize': '0.0',
  'cost': '-5057.8',
  'entryPrice': '0.19',
  'unrealizedPnl': '0.0',
  'realizedPnl': '-366.9188475',
  'initialMarginRequirement': '0.33333333',
  'maintenanceMarginRequirement': '0.03',
  'openSize': '26620.0',
  'collateralUsed': '1685.933316474',
  '

In [280]:
for pos in all_position:
    sym = (i['future'])
    last_price = get_ticker(sym)['last']
    pos['last'] = last_price
    pos['usd_value'] = abs(float(pos['cost']))
    
    
smart_order = pd.DataFrame(all_position)[['future','side','size','usd_value','last','recentBreakEvenPrice','recentPnl','estimatedLiquidationPrice']]
smart_order.columns = ['symbols','side','size','usd_value','last','entry_price','pnl','liquidate_price']

# Convert Str to Float 
smart_order[['entry_price','pnl','liquidate_price','size','last']] =smart_order[['entry_price','pnl','liquidate_price','size','last']].astype(float)
smart_order['diff_liq']=( smart_order['liquidate_price'] -smart_order['entry_price'] ) / smart_order['entry_price']
smart_order

,future,side,size,usd_value,last,recentBreakEvenPrice,recentPnl,estimatedLiquidationPrice
0,FTT-PERP,sell,61.8,3019.9188,0.18851,48.839,-1.6686,613.5693250439035
1,ZIL-PERP,sell,26620.0,5057.8000,0.18851,0.18951672802404207,-12.8647,0.5049557666803155


# Single Symbols

In [234]:
res_pos = get_position(symbol)
last_price = get_ticker(symbol)['last']
res_pos['last'] = last_price
# res_pos['usd_value'] = float(res_pos['recentBreakEvenPrice']) * float(res_pos['size'])
res_pos['usd_value'] = abs(float(res_pos['cost']))


smart_order = pd.DataFrame([res_pos])[['future','side','size','usd_value','last','recentBreakEvenPrice','recentPnl','estimatedLiquidationPrice']]
smart_order.columns = ['symbols','side','size','usd_value','last','entry_price','pnl','liquidate_price']
# Convert Str to Float 
smart_order[['entry_price','pnl','liquidate_price','size','last']] =smart_order[['entry_price','pnl','liquidate_price','size','last']].astype(float)
smart_order['diff_liq']=( smart_order['liquidate_price'] -smart_order['entry_price'] ) / smart_order['entry_price']
smart_order

,symbols,side,size,usd_value,last,entry_price,pnl,liquidate_price,diff_liq
0,ZIL-PERP,sell,26620.0,4823.2778,0.18128,0.189517,221.6575,0.507211,1.676339


In [ ]:
tp_targets = []
tp_sizes = []
for i in range(3):

    try:
        tp_target = input(f'TP  Target Percent {i+1}')
        tp_target = float(tp_target)

    except ValueError as e :
        print(str(e))
        break
    try:
        tp_size = input(f'TP    Target Size {i+1}')
        print('#'*100)

        tp_size = float(tp_size)

    except ValueError as e :
        print(str(e))
        break
        
    tp_targets.append(tp_target)
    tp_sizes.append(tp_size)
if sum(tp_sizes) != 1.0 :
    print('#'*100)
    print('Error  Size != 100% ')
    

In [289]:
sym_list = ['FTT-PERP','ZIL-PERP']

In [305]:
for i in smart_order.index:
    if smart_order.at[i,'future'] in sym_list:
        display(smart_order.iloc[[i,2:]])

SyntaxError: invalid syntax (<ipython-input-305-1156c5652dd3>, line 3)

In [310]:
smart_order.iloc[[0]]

,future,side,size,usd_value,last,recentBreakEvenPrice,recentPnl,estimatedLiquidationPrice
0,FTT-PERP,sell,61.8,3019.9188,0.18851,48.839,-1.6686,613.5693250439035


In [235]:
set_sl = True
set_tp = True
sl_trigger  = 0.10
tp_trigger  = 0.15

###
split_sl = False
split_tp = False


# tp_split =[0.05,0.02,0.03]
# sl_split = [0.02,0.03]
tp_split  =[]
sl_split =[]

for i in smart_order.index:
    first_trade =  smart_order.at[i,'usd_value'] - smart_order.at[i,'pnl']
    entry  =  smart_order.at[i,'entry_price']
    print('SELECTED SYMBOLS',smart_order.at[i,'symbols'],' Entry_price',entry,' Side : ',smart_order.at[i,'side']) 
    ## Check LIST TP , SL 
    if tp_split !=[]:
        # input list 
        tp_price_list = price_list(tp_split,entry,mode='tp',side= smart_order.at[i,'side'])
    else :
        tp_price_list = price_list([tp_trigger],entry,mode='tp',side= smart_order.at[i,'side'])

    if sl_split !=[]:
        sl_price_list = price_list(sl_split,entry,mode='sl',side= smart_order.at[i,'side'])
        
    else :
        sl_price_list = price_list([sl_trigger],entry,mode='sl',side= smart_order.at[i,'side'])
    
    if set_tp ==True:
        if split_tp ==True :
                for tp in range(len(tp_price_list)):
                    smart_order.at[i,f'tp_{tp+1}'] = tp_price_list[tp]

        else:
            smart_order.at[i,f'tp'] = tp_price_list[0] # Cal แยก side
        
    if set_sl == True:
        if split_sl == True:
            for sl in range(len(sl_price_list)):
                smart_order.at[i,f'sl_{sl+1}'] = sl_price_list[sl]
        else :
                smart_order.at[i,f'sl'] = sl_price_list[0] # cal แยก side
smart_order

SELECTED SYMBOLS ZIL-PERP  Entry_price 0.18951672802404207  Side :  sell
####################################################################################################


,symbols,side,size,usd_value,last,entry_price,pnl,liquidate_price,diff_liq,tp,sl
0,ZIL-PERP,sell,26620.0,4823.2778,0.18128,0.189517,221.6575,0.507211,1.676339,0.161089,0.208468


In [255]:
sum(tp_targets)

0.25

In [218]:
sl_price_list = price_list([sl_trigger],entry,mode='sl',side= smart_order.at[i,'side'])
sl_price_list

[0.20846840082644627]

# Below OLD WAYS

# Add New Function Futrue Bot 


In [686]:
def _future_postion_now():
    res= exchange.private_get_positions()

    list_trade = []
    for i in res['result']:
        if i['size']>0 :
            i['entry_p'] = (i['cost'] - i['realizedPnl']) /i['size']

            trade_dict = {}
            price= exchange.fetch_ticker(i['future'])
            trade_dict['symbols'] = (i['future'])
            trade_dict['side'] =i['side']
            trade_dict['size'] = i['size']
            trade_dict['entry_price'] = i['entry_p']
            trade_dict['last'] = i['entryPrice']
            trade_dict['diff'] = 1-(trade_dict['last'] / trade_dict['entry_price'])
            trade_dict['usd_values'] = trade_dict['entry_price'] * trade_dict['size']
            # add values 
            trade_dict['pnl'] = i['realizedPnl']
            trade_dict['init_margin'] = i['initialMarginRequirement']
            trade_dict['maintennance_margin'] = i['maintenanceMarginRequirement']
            trade_dict['collateral'] = i['collateralUsed']
            trade_dict['liquidate_price']  = i['estimatedLiquidationPrice']
#             print(trade_dict)
            list_trade.append(trade_dict)
    return list_trade

In [688]:
list_trade= _future_postion_now()
pos =pd.DataFrame(list_trade)
list_trade

[{'symbols': 'FTT-PERP',
  'side': 'buy',
  'size': 199.0,
  'entry_price': 62.01078364773869,
  'last': 61.274,
  'diff': 0.011881540667573187,
  'usd_values': 12340.1459459,
  'pnl': -146.6199459,
  'init_margin': 0.1,
  'maintennance_margin': 0.03,
  'collateral': 1219.3526,
  'liquidate_price': 45.56976442123139},
 {'symbols': 'MATIC-PERP',
  'side': 'buy',
  'size': 7203.0,
  'entry_price': 1.542006809557129,
  'last': 1.5255425,
  'diff': 0.01067719640087561,
  'usd_values': 11107.07504924,
  'pnl': -118.59242174,
  'init_margin': 0.1,
  'maintennance_margin': 0.03,
  'collateral': 1098.84826275,
  'liquidate_price': 1.0916756556053098}]

In [689]:
smart_trade = []
if list_trade != []:
    for trade in list_trade:
        if trade['symbols'] == 'MATIC-PERP':
            print('OK')
            smart_trade.append(trade) 
        else:
            print('Order Not found Want To New Trades?')

Order Not found Want To New Trades?
OK


In [694]:
# Raw 
smart_order  =pd.DataFrame(list_trade) # all symbols
smart_order['diff_liq']=( smart_order['liquidate_price'] -smart_order['entry_price'] ) / smart_order['entry_price']
smart_order

,symbols,side,size,entry_price,last,diff,usd_values,pnl,init_margin,maintennance_margin,collateral,liquidate_price,diff_liq
0,FTT-PERP,buy,199.0,62.010784,61.260000,0.012107,12340.145946,-149.405946,0.1,0.03,1219.07400,45.603665,-0.264585
1,MATIC-PERP,buy,7203.0,1.542007,1.524565,0.011311,11107.075049,-125.633354,0.1,0.03,1098.14417,1.092022,-0.291818


In [692]:
# Approve Sell side 
def smart_trade(symbols,size,price):
    print(symbols,size,price)
    return 

split_sl = False
split_tp = False

list_trade= _future_postion_now()
# smart_order  =pd.DataFrame(smart_trade) # Selecte symbols
smart_order  =pd.DataFrame(list_trade) # all symbols
smart_order['diff_liq']=( smart_order['liquidate_price'] -smart_order['entry_price'] ) / smart_order['entry_price']

for i in smart_order.index:
    print('SELECTED SYMBOLS',smart_order.at[i,'symbols']) 
    entry  =  smart_order.at[i,'entry_price']
    if smart_order.at[i,'side'] == 'buy':
        
        first_trade =  smart_order.at[i,'usd_values'] - smart_order.at[i,'pnl']
    if set_tp == True:
        if split_tp == True:
            tp_price_list = price_list(tp_split,entry,mode='tp')
            for tp in range(len(tp_price_list)):
                smart_order.at[i,f'tp_{tp+1}'] = tp_price_list[tp]
                smart_order.at[i,f'tp_{tp+1}_$']=(smart_order.at[i,f'tp_{tp+1}']  *smart_order.at[i,'size'] ) - first_trade 
        else :
            smart_order.at[i,f'tp'] = (entry * tp_trigger)+entry
            smart_order.at[i,f'tp_$']=(smart_order.at[i,f'tp']  *smart_order.at[i,'size'] ) - first_trade 

############################################################################################################            
    if set_sl == True:
        if split_sl == True:
            sl_price_list = price_list(sl_split,entry,mode='sl')
            for sl in range(len(sl_price_list)):
                smart_order.at[i,f'sl_{sl+1}'] = sl_price_list[sl]
                smart_order.at[i,f'sl_{sl+1}_$']=(smart_order.at[i,f'sl_{sl+1}']  *smart_order.at[i,'size'] ) - first_trade 

        else :
            smart_order.at[i,f'sl'] = entry - (entry * sl_trigger)
            smart_order.at[i,f'sl_$']=(smart_order.at[i,f'sl']  *smart_order.at[i,'size'] ) - first_trade 


SELECTED SYMBOLS FTT-PERP
SELECTED SYMBOLS MATIC-PERP


In [693]:
smart_order = smart_order[smart_order.columns[~smart_order.columns.isin(['maintennance_margin','collateral','init_margin'])]]
smart_order
# for loop send nortify price check tp ,sl ,diff 

,symbols,side,size,entry_price,last,diff,usd_values,pnl,liquidate_price,diff_liq,tp,tp_$,sl,sl_$
0,FTT-PERP,buy,199.0,62.010784,61.260000,0.012107,12340.145946,-149.405946,45.603665,-0.264585,68.211862,1084.608649,58.910244,-766.413243
1,MATIC-PERP,buy,7203.0,1.542007,1.524565,0.011311,11107.075049,-125.633354,1.092022,-0.291818,1.696207,985.074151,1.464906,-680.987107


In [666]:
def tp_split_clean(tp_split:list):
#     tp_split = [0.03,0.02,0.05]
    try:
    # Fill Loop with 3 array
        while len(tp_split) != 3:
            tp_split.append(0.00)

        return tp_split
    except:
        print('Values Error ')

def sl_split_clean(sl_split:list):
    try:

        while len(sl_split) != 3:
            sl_split.append(0.00)
            return sl_split
    except :
        print('Values Error ')
def price_list(list_split,entry_price,mode='tp'):
    sum_p = 0.0
    price_list = []
    if mode== 'tp':
        for i in list_split:
            sum_p +=i 
            tp_price =(entry_price * sum_p)+ entry_price
            price_list.append(tp_price)
        price_list = list(set(price_list))
#     elif mode == 'sl':
    else:
        for i in list_split:
            sum_p +=i 
            sl_price = entry_price - (entry_price * sum_p)
            price_list.append(sl_price)
            price_list = list(set(price_list))
        
    return price_list

        
        
tp_split =tp_split_clean([0.05,0.02,0.03])
sl_split = sl_split_clean([0.02,0.03])



tp_price_list = price_list(tp_split,61.119,mode='tp')
sl_price_list = price_list(sl_split,61.119,mode='sl')

print(tp_split,sl_split)
print(tp_price_list,sl_price_list)

[0.05, 0.02, 0.03] [0.02, 0.03, 0.0]
[64.17495, 65.39733, 67.2309] [58.06305, 59.89662]
